In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
import torch
from collections import defaultdict
import re

# 🔧 경로 설정
ADAPTER_PATH = "../Models/ToneDetect_adapter"  # adapter만 있으면 됨
BASE_MODEL_NAME = "beomi/kcbert-base"          # 학습 시 사용한 base model 이름

# ✅ tokenizer는 base model 기준
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# ✅ base model → adapter 연결
base_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_NAME, num_labels=5)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

# ✅ 라벨 매핑 정의
label_map = {
    0: "chat_emoticon(이모티콘 자주 쓰는 말투)",
    1: "elder_speech(어르신 말투)",
    2: "formal(격식있는 말투)",
    3: "informal(친근한 말투)",
    4: "soft_polite(부드럽고 상냥한 말투)"
}
reverse_label_map = {v: k for k, v in label_map.items()}

# ✅ 사용자별 메시지 분리
def parse_chat_file(filename):
    user_messages = defaultdict(list)
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if re.match(r"^-{5,}", line) or re.match(r"^\d{4}년 \d{1,2}월 \d{1,2}일", line):
                continue
            match = re.match(r"\[([^]]+)] \[([^]]+)] (.+)", line)
            if match:
                user, time, message = match.groups()
                user_messages[user].append(message)
    return user_messages

# ✅ 문장 하나 예측
def predict_style(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_class = torch.argmax(logits, dim=1).item()
    return pred_class

# ✅ 전체 채팅 예측 분석
def analyze_chat_styles_with_messages(filename):
    user_messages = parse_chat_file(filename)
    result_summary = {}
    result_sentences = {}

    for user, messages in user_messages.items():
        style_counts = defaultdict(int)
        style_messages = defaultdict(list)
        for message in messages:
            predicted_class = predict_style(message)
            style_name = label_map[predicted_class]
            style_counts[style_name] += 1
            style_messages[style_name].append(message)
        result_summary[user] = dict(style_counts)
        result_sentences[user] = dict(style_messages)

    return result_summary, result_sentences

# ✅ 사용자 말투 별 문장 보기
def get_sentences_by_style(user, style_name, result_sentences):
    if user not in result_sentences:
        print(f"[오류] 사용자 '{user}'를 찾을 수 없습니다.")
        return
    if style_name not in result_sentences[user]:
        print(f"[알림] '{user}'는 '{style_name}' 말투로 분류된 문장이 없습니다.")
        return
    print(f"\n[{user}]의 '{style_name}' 예측 문장들:")
    for idx, sentence in enumerate(result_sentences[user][style_name], 1):
        print(f"{idx}. {sentence}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:

# ✅ 실행 예시
filename = "../datasets/KakaoTalk_20250515_0053_22_930_유정유정.txt"
result_summary, result_sentences = analyze_chat_styles_with_messages(filename)

# ✅ 사용자별 결과 요약 출력
print("\n📊 [전체 사용자별 예측 결과 요약]")
for user, style_count in result_summary.items():
    print(f"\n🧑 {user}")
    for style, count in style_count.items():
        print(f"  - {style}: {count}문장")



📊 [전체 사용자별 예측 결과 요약]

🧑 김소연
  - elder_speech(어르신 말투): 2문장
  - soft_polite(부드럽고 상냥한 말투): 3문장
  - chat_emoticon(이모티콘 자주 쓰는 말투): 16문장
  - informal(친근한 말투): 3문장
  - formal(격식있는 말투): 2문장

🧑 유정유정
  - chat_emoticon(이모티콘 자주 쓰는 말투): 22문장
  - elder_speech(어르신 말투): 1문장
  - informal(친근한 말투): 5문장
  - formal(격식있는 말투): 2문장
  - soft_polite(부드럽고 상냥한 말투): 3문장


In [23]:

# ✅ 원하는 예측 문장 보기
get_sentences_by_style("김소연", "soft_polite(부드럽고 상냥한 말투)", result_sentences)
get_sentences_by_style("유정유정", "chat_emoticon(이모티콘 자주 쓰는 말투)", result_sentences)


[김소연]의 'soft_polite(부드럽고 상냥한 말투)' 예측 문장들:
1. 하던게 안끝났어.. 쏘리..
2. 아 헐 나 오늘 계속 자버렸네.. 지금부터 하고 있을게ㅠㅠ
3. 나도 계속 해보고 있어..!!

[유정유정]의 'chat_emoticon(이모티콘 자주 쓰는 말투)' 예측 문장들:
1. 헉 아냐아냐!!?
2. 여유있게 와 ㅎㅎ
3. 30분쯤 보쟝!!
4. 헉 !!! 나는 버스가 늦어져서 곧 탈 거 같아...ㅎㅎ
5. 그 스타벅스 갈까 하는데 어때??
6. [네이버 지도]
7. 웅웅 고마어🥰🥰
8. 언능 갈게!!!
9. 나 내렸오 ㅎㅎ
10. 스벅 앞이양??
11. 건너편에 투썸 있는데 가볼까???
12. 건너 올랭?????
13. 알써😆😆
14. https://www.genspark.ai/
15. 언니 진짜 너무 고샹해써... 조심히 잘 들어가구 화욜에 보자😂😂❤
16. 오케이!! 고마웡🥰
17. 언니 !! 나 대본 다썼어 ㅎㅎ
18. 오 진짜??? 조아조아 고마어 ㅎㅎㅎㅎ
19. 내일 이대로 할겡 ㅎㅎ
20. 언니 혹시 코드 분석 얼마나 했어?? 내가 아직 학교라서 집 가서 할 수 있을 거 같아….
21. 에...? 진짜 괜찮게써?? 내 오움이 필요ㅘ면 꼭 말해줘!!!!!
22. 이따보쟝,, 진짜 고맙구 수고해써😂❤❤


In [ ]:
# 파일 입력 받기 (KakaoTalk_20250511_1625_29_658_유정유정)

# 파일의 형식은 아래와 같음
# --------------- 2025년 3월 19일 수요일 ---------------
# [유정유정] [오후 2:02] 언니 혹시 끝났어??
# [김소연] [오후 2:03] 아직 수업중ㅠㅠ
# [유정유정] [오후 2:03] 헉!!
# [유정유정] [오후 2:03] 먼저 가고 있을게 ㅎㅎ
# [김소연] [오후 2:03] 웅웅~
# [유정유정] [오후 2:08] 오른쪽 앞에서 두번째 안쪽으로 자리 잡아놔쓰😆❤
# [김소연] [오후 2:16] 나 방금 끝나서 가는 중!
# --------------- 2025년 3월 26일 수요일 ---------------
# [유정유정] [오후 2:16] ㅎㅎ 언니 미안... 또 앞자리얌...
# [유정유정] [오후 2:16] 저번에 앉았던 그 자리...^^

# 날짜 부분 버리기
# 이름 별로 문장 모으기

# 기존 코드와 합치기
# 최종 결과는 이름 당 각 클래스로 예측된 문장 갯수
# 예시)
# 김소연 -> {로봇 말투 (android): 5, 연장자 말투(azae):3 ...}
# 유정유정 -> {로봇 말투 (android): 2, 연장자 말투(azae):7 ...}


In [ ]:
text = "안녕하세요"  # 예측하고 싶은 문장

# 입력 토크나이즈
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# 예측
with torch.no_grad():
    outputs = model(**inputs)   
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

label_map = {
    0: "chat_emoticon",
    1: "elder_speech",
    2: "formal",
    3: "informal",
    4: "soft_polite"
}

print("예측된 클래스:", predicted_class)
print("예측된 말투:", label_map[predicted_class])


예측된 클래스: 6
예측된 말투: 존댓말 (formal)
